In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init
from torch.utils.data import DataLoader

device = 'cpu'
torch.manual_seed(333)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='/tmp/pycharm_project_001/MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='/tmp/pycharm_project_001/MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

data_loader = DataLoader(dataset=mnist_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [2]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2)
                                    )

        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2)
                                    )

        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2)
                                    )
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)     # Flatten them for FC
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [3]:
model = CNN().to(device)
value = torch.Tensor(1, 1, 28, 28).to(device)
print(model(value).shape)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

torch.Size([1, 10])


In [4]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost /total_batch
    print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print('Learning finished')

[Epoch :    1] cost = 0.159101307
[Epoch :    2] cost = 0.0431748964
[Epoch :    3] cost = 0.028897427
[Epoch :    4] cost = 0.0230954979
[Epoch :    5] cost = 0.0178326853
[Epoch :    6] cost = 0.0148608685
[Epoch :    7] cost = 0.012440471
[Epoch :    8] cost = 0.0126087936
[Epoch :    9] cost = 0.00823160633
[Epoch :   10] cost = 0.00890754908
[Epoch :   11] cost = 0.00864369888
[Epoch :   12] cost = 0.00622282876
[Epoch :   13] cost = 0.00565810874
[Epoch :   14] cost = 0.00670281565
[Epoch :   15] cost = 0.00575227849
Learning finished


In [5]:
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.9879000186920166
